In [41]:
import requests
import os
import json
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd

In [121]:
download_location = "C:/Users/Lara/Work/DataDownloads/testData/"
temp_download_location = "C:/Users/Lara/Work/DataDownloads/testDataTemp/"
lookup_location = "C:/Users/Lara/Work/DataDownloads/Lookups/"
item_url_file = "ItemURL_ArcGIS_6monthUpdates.csv"
boundary_file = "C:/Users/Lara/Work/DataDownloads/CountyBoundary.shp"

In [122]:
boundary_gdf = gpd.read_file(boundary_file)

In [3]:
# Read the csv file
item_url_df = pd.read_csv(lookup_location + item_url_file)
item_url_df.head(3)

,Dataset,Owner,URL
0,Local Nature Reserves (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
1,National Nature Reserves (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
2,Priority Habitats Inventory (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...


In [123]:
#Loop through each dataset name in lookup
for url in item_url_df.URL:
    print(url)
    # Get url that contains details for dataset
    url_details = url.replace("/query?", "?f=json")
    
    # Get dataset name and source from lookup
    dataset = item_url_df[item_url_df.URL==url]['Dataset'].item() 
    source = item_url_df[item_url_df.URL==url]['Owner'].item()

    ## DOwnload dataset
    params = {
    'where': "1=1",  # Select all data
    'outFields': "*",  # All fields
    'f': 'json',  # Return format
    # 'geometryType': 'esriGeometryPoint',  # Can change based on geometry type
    'outSR': '27700',  # Coordinate system. Unsure if needed, from documentation: If outSR is not specified, the geometry is returned in the spatial reference of the map.
    }
    response_dataset = requests.get(url, params=params)

    # CHeck request worked
    if response_dataset.status_code == 200:
        # Download geojson 
        with open(f"{temp_download_location}{dataset}.geojson", 'wb') as file:
            file.write(response_dataset.content)
        #Create folder if not already there
        if not os.path.exists(f"{download_location}Original/{source}/{dataset}"):
            os.makedirs(f"{download_location}Original/{source}/{dataset}")
        if not os.path.exists(f"{download_location}Wiltshire/{source}/{dataset}"):
            os.makedirs(f"{download_location}Wiltshire/{source}/{dataset}")
            
        # Need to read geojson back in then write out the shapefile, as arcgis does not read geojson
        gdf = gpd.read_file(f"{temp_download_location}{dataset}.geojson")

        gdf.to_file(f"{download_location}Original/{source}/{dataset}/{dataset}.gpkg")

        gdf_wilts = gpd.clip(gdf, boundary_gdf)
        gdf_wilts.to_file(f"{download_location}Wiltshire/{source}/{dataset}/{dataset}.gpkg")
    else:
        print(f"Request failed for {dataset}")

    # Download dataset details
    response_details = requests.get(url_details)
    if response_details.status_code == 200:
        details = response_details.json()
        print(details['editingInfo']['dataLastEditDate'])
    else:
        print(f"Request failed for details of {dataset}")

    

https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Local_Nature_Reserves_England/FeatureServer/0/query?


KeyboardInterrupt: 